In [25]:
import pandas as pd
from binance.client import Client
from Binance_api import api_key, secret_key
from datetime import datetime
from datetime import timedelta
import time
from binance.exceptions import BinanceAPIException
import pandas as pd
import numpy as np
import os
from binance.client import Client
from Binance_api import api_key, secret_key
import time
import plotly.graph_objects as go
import talib as ta
from datetime import datetime
import os
from ta import add_all_ta_features
import warnings
import sklearn
from sklearn.decomposition import PCA
import plotly.express as px
import numpy
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.momentum import RSIIndicator
from knockknock import email_sender
import smtplib 
from email.message import EmailMessage
import math
from os import truncate

warnings.filterwarnings('ignore')

client = Client(api_key, secret_key,tld='us')

rsi = 0
bottoms_5 = [9999999,99999999,99999999,99999999,9999999]
rsi_l = [9999999,9999999,999999999,999999999,99999999]

buy = 0
sell = 0

In [26]:
def sma(data, window):
    return(data.rolling(window = window).mean())

def bollinger_band(data, sma, window, nstd):
    std = data.rolling(window = window).std()
    upper_band = sma + std * nstd
    lower_band = sma - std * nstd
    return upper_band, lower_band
nstd = 2
def rsi(symbol, i):
    if df[f'{symbol}_rsi'].iloc[i] > rsi:
        rsi = df[f'{symbol}_rsi'].iloc[i]
    return rsi

def bottoms(new_bottom,rsi_new, bottom_5, rsi_l):
    is_new = 0
    for a in range(len(bottoms_5)):
        if new_bottom < bottoms_5[a]:
            is_new = 1
    if is_new == 1:
        br = len(bottoms_5) - 1
        while br > 0:
            bottoms_5[br] = bottoms_5[br-1]
            rsi_l[br] = rsi_l[br-1]
            br -= 1
        bottoms_5[0] = new_bottom
        rsi_l[0] = rsi_new
    return bottoms_5,rsi_l

def divergence(df,i,symbol):
    p = i - 1
    while p > -150:
        if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}-USD_Low'].iloc[p] and df[f'{symbol}_rsi'].iloc[i] > df[f'{symbol}_rsi'].iloc[p]:
            return True
        else: 
            p = p - 1

def email_alert(subject, body, to):
    msg = EmailMessage()
    msg.set_content(body)
    msg['subject'] = subject
    msg['to'] = to
    


    user = "lcolesemailalertbot@gmail.com"
    msg['from'] = user
    password = "lyxzpdpvklyyjvkp"

    server = smtplib.SMTP("smtp.gmail.com", 587)
    server.starttls()
    server.login(user, password)
    server.send_message(msg)

    server.quit()


    


In [27]:
email_alert("Bitcoin", "Program Started", "4fqxg1hz19@pomail.net")
symbols = ['BTC']
start_n_hours_ago = 48
def gather_data(symbols, start_n_hours_ago):
        merge = False
        for symbol in symbols:
                klines = client.get_historical_klines(symbol=f'{symbol}USDT', interval=client.KLINE_INTERVAL_5MINUTE, start_str=str(datetime.now()-timedelta(hours=start_n_hours_ago)))
                
                cols = ['OpenTime',
                        f'{symbol}-USD_Open',
                        f'{symbol}-USD_High',
                        f'{symbol}-USD_Low',
                        f'{symbol}-USD_Close',
                        f'{symbol}-USD_Volume',
                        'CloseTime',
                        f'{symbol}-QuoteAssetVolume',
                        f'{symbol}-NumberOfTrades',
                        f'{symbol}-TBBAV',
                        f'{symbol}-TBQAV',
                        f'{symbol}-ignore']
                
                df = pd.DataFrame(klines,columns=cols)

                if merge == True:
                        dfs = pd.merge(df,dfs,how='inner',on=['OpenTime','CloseTime'])
                else:
                        dfs = df
                        merge = True
        dfs['OpenTime'] = [datetime.fromtimestamp(ts / 1000) for ts in dfs['OpenTime']]
        dfs['CloseTime'] = [datetime.fromtimestamp(ts / 1000) for ts in dfs['CloseTime']]

        for col in dfs.columns: 
                if not 'Time' in col:
                        dfs[col] = dfs[col].astype(float)

        for symbol in symbols:
                dfs[f'{symbol}_sma'] = sma(dfs[f'{symbol}-USD_Close'],window=20)
                dfs[f'{symbol}_upper_band'], dfs[f'{symbol}_lower_band'] = bollinger_band(data=dfs[f'{symbol}-USD_Close'], sma=dfs[f'{symbol}_sma'], window=20, nstd=3)
                #dfs[f'{symbol}_rsi'] = RSIIndicator(dfs[f'{symbol}-USD_Close'], window = 14, fillna = False)
                
                dfs[f'{symbol}_rsi'] = ta.RSI(dfs[f'{symbol}-USD_Close'])

        dfs.dropna(inplace=True)

        return dfs

def get_states(df, symbols):
    states = {}

    for symbol in symbols:
        if df[f'{symbol}-USD_Close'].iloc[-1] < df[f'{symbol}_lower_band'].iloc[-1]:
            states[symbol] = 'below'
        elif df[f'{symbol}-USD_Close'].iloc[-1] > df[f'{symbol}_upper_band'].iloc[-1]:
            states[symbol] = 'above'
        else:
            states[symbol] = 'inside'

df = gather_data(symbols,48)


states

In [18]:
client.get_orderbook_ticker(symbol=f'BTCUSDT')

{'symbol': 'BTCUSDT',
 'bidPrice': '16463.62000000',
 'bidQty': '0.04740100',
 'askPrice': '16469.50000000',
 'askQty': '0.17071400'}

In [28]:
Buy_amount = 50
balance_unit = 'USDT'
last_buy = 0
First = True
program_start = datetime.now()
passed = 1
precision = client.get_symbol_info('BTCUSDT')['quotePrecision']
while True:
    if (datetime.now() - program_start).seconds == 1800*passed:
        email_alert("Bitcoin", "Still running", "4fqxg1hz19@pomail.net")
        passed = passed + 1
    if (datetime.now().second % 10 == 0) or First:
        if (datetime.now().minute == 0 and datetime.now().second == 10) or First:
            #refresh data
            first = False
            df = gather_data(symbols,48)
            states = get_states(df, symbols)
            #print('current state of the market:')
            #print(states)
    try:
        if balance_unit == 'USDT': #looking to buy
            for symbol in symbols:
                ask_price = float(client.get_orderbook_ticker(symbol=f'{symbol}USDT')['askPrice'])
                lower_band = df[f'{symbol}_lower_band'].iloc[-1]
                if ask_price < lower_band:
                    bottoms_5,rsi_l = bottoms(ask_price, df[f'{symbol}_rsi'].iloc[-1])
                    if divergence(df,-1,symbols):
                        buy = 1
                        email_alert("Bitcoin", "Divergence", "4fqxg1hz19@pomail.net")
                        print("divergence")
                if buy == 1 and df[f'{symbol}-USD_Close'].iloc[-1] > df[f'{symbol}-USD_Open'].iloc[-1]:
                    email_alert("Bitcoin", "Buy", "4fqxg1hz19@pomail.net")
                    buy_order = client.order_limit_buy(symbol='BTCUSDT', quantity= truncate(Buy_amount/ ask_price, precision), price = ask_price)
                    start = datetime.now()
                    while True:
                        time.sleep(1)
                        buy_order = client.get_order(symbol=buy_order['symbol'], orderId=buy_order[ 'orderID'])  
                        seconds_since_buy = (datetime.now() - start).seconds 
                        if float(buy_order['executeQty']) == 0 and seconds_since_buy > 3600:
                            #nofill
                            client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
                            break

                        if float(buy_order['executedQty']) != 0 and float(buy_order['executedQty']) != float(buy_order['origQty']) and seconds_since_buy > 3600:
                            #partial fill
                            client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
                            balance_unit = symbol
                            last_buy = ask_price
                            buy = 0
                            break
                        
                        if float(buy_order['executedQty']) == float(buy_order['origQty']):
                            #completely filled 
                            blance_unit = symbol 
                            last_buy = ask_price
                            buy = 0
                            balance_unit = symbol
                            break 
        
        if balance_unit != 'USDT': #looking to sell
            bid_price = float(client.get_orderbook_ticker(symbol= f'{balance_unit}USDT')['bidPrice'])
            upper_band = df[f'{symbol}_upper_band'].iloc[-1]
            if last_buy*0.98 > ask_price:
                email_alert("Bitcoin", "2 percent loss, SELL", "4fqxg1hz19@pomail.net")
                client.order_market_sell(symbol=['symbol'],quantity=truncate(buy_order['executedQty'], precision))
                sell = 0
            elif ask_price > upper_band and df[f'{symbol}_rsi'].iloc[-1] > 68:
                sell = 1
                email_alert("Bitcoin", "sell on next red candle", "lcoles2812@gmail.com")
            if sell == 1 and df[f'{symbol}-USD_Close'].iloc[-1] < df[f'{symbol}-USD_Open'].iloc[-1]:
                email_alert("Bitcoin", "TIME TO SELL", "4fqxg1hz19@pomail.net")
                client.order_market_sell(symbol=['symbol'],quantity=truncate(buy_order['executedQty'], precision))
                sell = 0
        time.sleep(1)

    except BinanceAPIException as e:
        print(e.status_code)
        print(e.message)

        

KeyboardInterrupt: 

In [ ]:
start = datetime.now()
while True:
    time.sleep(1)
    buy_order = client.get_order(symbol=buy_order['symbol'], orderId=buy_order['orderID'])  
    seconds_since_buy = (datetime.now() - start).seconds 
    if float(buy_order['executeQty']) == 0 and seconds_since_buy > 3600:
        #nofill
        client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
        break

    if float(buy_order['executedQty']) != 0 and float(buy_order['executedQty']) != float(buy_order['origQty']) and seconds_since_buy > 3600:
        #partial fill
        client.cancel_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])
        balance_unit = symbol
        break
    
    if float(buy_order['executedQty']) == float(buy_order['origQty']):
        #completely filled 
        blance_unit = symbol 
        break 

In [ ]:
buy_order = client.order_limit_buy(symbol='BTCUSDT', quantity= Buy_amount/ ask_price, price = 55000)

{'BTC': 8}

In [46]:
try:
    client.get_all_orders()
except BinanceAPIException as e:
    print(e.status_code)
    print(e.message)


NameError: name 'BinanceAPIException' is not defined

In [11]:
env = TradingEnv(balance_amount=100,balance_unit='USDT', trading_fee_multiplier=0.99925, symbols=symbols)

for i in range(len(df)):  
    #means looking for something to buy
    if env.balance_unit == 'USDT':
        for symbol in symbols:
            if env.bottoms[symbol] == 'hit' and df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]:
                env.bottoms[symbol] = 'released'
            
            if df[f'{symbol}-USD_Low'].iloc[i] < df[f'{symbol}_lower_band'].iloc[i]: #buy signal
                if env.bottoms[symbol] == 'released':
                    env.buy(symbol, df[f'{symbol}_lower_band'].iloc[i], df['OpenTime'].iloc[i])
                    env.reset_bottoms()
                    break
                else:
                    env.bottoms[symbol] = 'hit'
    
    #means looking to sell
    if env.balance_unit != 'USDT':
        if env.tops[env.balance_unit] == 'hit' and df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]:
            env.tops[env.balance_unit] = 'released'
        
        if df[f'{env.balance_unit}-USD_High'].iloc[i] > df[f'{env.balance_unit}_upper_band'].iloc[i]: #sell signal
            if env.tops[env.balance_unit] == 'released':
                env.sell(df[f'{env.balance_unit}_upper_band'].iloc[i], df['OpenTime'].iloc[i])
                env.reset_tops()
            else:
                env.tops[env.balance_unit] = 'hit'

if env.balance_unit != 'USDT':
    env.sell(df[f'{env.balance_unit}-USD_Close'].iloc[-1], df['OpenTime'].iloc[-1])
    
   

In [12]:
print(f'num buys: {len(env.buys)}')
print(f'num sells: {len(env.sells)}')
print(f'ending balance: {env.balance_amount} {env.balance_unit}')

num buys: 70
num sells: 70
ending balance: 89.33740476289911 USDT


In [31]:
for i in range(len(env.buys)):
    buys = env.buys[i]
    sells = env.sells[i]
    profit = ((sells[2]/buys[2])-1)*100
    print(buys)
    print(sells)
    print('P/L = 'f'{profit}')
    print('\n')

['BTC', '2022-11-16 10:56:00', 16792.25083691415]
['BTC', '2022-11-16 11:57:00', 16855.338406492894]
P/L = 0.37569454024626303


['BTC', '2022-11-16 14:22:00', 16928.967577869214]
['BTC', '2022-11-16 15:18:00', 16922.039155381153]
P/L = -0.0409264324962022


['BTC', '2022-11-16 16:11:00', 16889.292258865134]
['BTC', '2022-11-16 17:02:00', 16896.161880173106]
P/L = 0.04067441786594905


['BTC', '2022-11-16 18:00:00', 16815.619922646805]
['BTC', '2022-11-16 20:04:00', 16773.04289935849]
P/L = -0.2531992485806267


['BTC', '2022-11-16 20:36:00', 16691.752089485755]
['BTC', '2022-11-16 20:43:00', 16754.65165393053]
P/L = 0.37683020995977756


['BTC', '2022-11-16 21:49:00', 16709.191793541515]
['BTC', '2022-11-16 23:33:00', 16674.15172413135]
P/L = -0.20970535165986126


['BTC', '2022-11-16 23:56:00', 16474.124285860224]
['BTC', '2022-11-17 00:34:00', 16578.37796267964]
P/L = 0.6328328899940061


['BTC', '2022-11-17 01:46:00', 16490.00050053416]
['BTC', '2022-11-17 02:41:00', 16505.97528804

In [ ]:
start = datetime.now()
while True:
    time.sleep(1)
    buy_order = client.get_order(symbol=buy_order['symbol'], orderId=buy_order['orderId'])

    seconds_since_buy = datetime.now() - start.seconds
    print(seconds_since_buy)
    if float(buy_order['executedQty']) == 0 and seconds_since_buy > 3600:
        client.cancel_order()

